# Testing diffusion problem format

Grid and parameters

In [1]:
struct Grid
    xmax::Float64
    ymax::Float64

    nx::Int64
    ny::Int64
    
    Δx::Float64
    Δy::Float64

    xRange::Array{Float64,1}
    yRange::Array{Float64,1}

    function Grid(xmax, ymax, nx, ny)
        Δx = 2*xmax/(nx-1)
        Δy = 2*ymax/(ny-1)

        xRange = (-xmax:Δx:xmax)
        yRange = (-ymax:Δy:ymax)

        return new(xmax, ymax, nx, ny, Δx, Δy, xRange, yRange)
    end
end
struct Parameters
    ν::Float64
    σ::Float64
    tspan::Tuple{Float64,Float64}
end

Model setup

In [2]:
abstract type FluidModel end
mutable struct DiffusionModel <: FluidModel
    u::Array{Float64,2}
    
    grid::Grid
    parameters::Parameters
    
    Δt::Float64
    t::Array{Float64,1}
    function DiffusionModel(grid, parameters)
        u = zeros(grid.nx, grid.ny)
        
        Δt = parameters.σ*grid.Δx*grid.Δy/parameters.ν
        t = parameters.tspan[1]:Δt:parameters.tspan[2]
        
        return new(u, grid, parameters, Δt ,t)
    end
end

Test setup of model

In [3]:
grid_test = Grid(1., 2., 4, 6)
param_test = Parameters(0.2, 0.00025, (0., 1.))
mod_test = DiffusionModel(grid_test, param_test)
dump(mod_test)

DiffusionModel
  u: Array{Float64}((4, 6)) [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
  grid: Grid
    xmax: Float64 1.0
    ymax: Float64 2.0
    nx: Int64 4
    ny: Int64 6
    Δx: Float64 0.6666666666666666
    Δy: Float64 0.8
    xRange: Array{Float64}((4,)) [-1.0, -0.3333333333333333, 0.3333333333333333, 1.0]
    yRange: Array{Float64}((6,)) [-2.0, -1.2, -0.4, 0.4, 1.2, 2.0]
  parameters: Parameters
    ν: Float64 0.2
    σ: Float64 0.00025
    tspan: Tuple{Float64,Float64}
      1: Float64 0.0
      2: Float64 1.0
  Δt: Float64 0.0006666666666666666
  t: Array{Float64}((1501,)) [0.0, 0.0006666666666666666, 0.0013333333333333333, 0.002, 0.0026666666666666666, 0.0033333333333333335, 0.004, 0.004666666666666667, 0.005333333333333333, 0.006  …  0.994, 0.9946666666666667, 0.9953333333333333, 0.996, 0.9966666666666667, 0.9973333333333333, 0.998, 0.9986666666666667, 0.9993333333333333, 1.0]


Methods

In [4]:
abstract type DiffusionMethod end
struct ExplicitDiffusion <: DiffusionMethod end
struct CrankNickolsonDiffusion <: DiffusionMethod end
struct ADIDiffusion <: DiffusionMethod end

In [15]:
function perform_step!(model::DiffusionModel, method::ExplicitDiffusion, cache::ExplicitDiffusionCache)
    #un = cache.uprev
    #nx = integrator.grid.nx
    #ny = integrator.grid.ny
    #Δx = integrator.grid.Δx
    #Δy = integrator.grid.Δy
    #Δt = integrator.parameters.Δt
    #ν  = integrator.parameters.ν
    println("t = ", cache.t)
    #copy!(un,u)
    #for j in 2:ny-1, i in 2:nx-1
    #    u[j,i] = un[j,i] + 
    #        ν*Δt/(Δx*Δx)*(un[j  ,i+1]-2*un[j,i]+un[j  ,i-1]) + 
    #        ν*Δt/(Δy*Δy)*(un[j+1,i  ]-2*un[j,i]+un[j-1,i  ])
    #end

    # BCs
    #u[1,:]   .= 0.0
    #u[end,:] .= 0.0
    #u[:,1]   .= 0.0
    #u[:,end] .= 0.0
end;

LoadError: UndefVarError: ExplicitDiffusionCache not defined

Cache

In [16]:
abstract type DiffusionCache end
mutable struct ExplicitDiffusionCache <: DiffusionCache
    u::Array{Float64,2}
    uprev::Array{Float64,2}
    t::Float64
end

LoadError: invalid redefinition of constant DiffusionCache

Problem = model + method + (cache)

In [7]:
abstract type FluidProblem end
mutable struct DiffusionProblem{MethodType<:DiffusionMethod} <: FluidProblem
    model::DiffusionModel
    method::MethodType
    t::Float64
    
    #function DiffusionProblem(model, method)
    #    t = model.parameters.tspan[1]
    #    
    #    return new{MethodType<:DiffusionMethod}(model, method, t)
    #end
end
function DiffusionProblem(model, M::MethodType) where MethodType
    t = model.parameters.tspan[1]
    return DiffusionProblem{MethodType}(model, M, t)
end;
#DiffusionProblem(model, M::MethodType, t) where MethodType = 
#        DiffusionProblem{MethodType}(model, M, t)

Solve problem

In [8]:
function solve(prob::DiffusionProblem)
    solve!(prob)
    return integrator.sol
end
function solve!(prob::DiffusionProblem)
    while prob.t < prob.model.parameters.tspan[2]
        timestep!(prob)
    end
end
function timestep!(prob::DiffusionProblem)
    perform_step!(prob.model, prob.method)
    prob.t += prob.model.Δt
end;

Initialise problem object

In [9]:
prob_test = DiffusionProblem(mod_test, ExplicitDiffusion())
dump(prob_test)

DiffusionProblem{ExplicitDiffusion}
  model: DiffusionModel
    u: Array{Float64}((4, 6)) [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
    grid: Grid
      xmax: Float64 1.0
      ymax: Float64 2.0
      nx: Int64 4
      ny: Int64 6
      Δx: Float64 0.6666666666666666
      Δy: Float64 0.8
      xRange: Array{Float64}((4,)) [-1.0, -0.3333333333333333, 0.3333333333333333, 1.0]
      yRange: Array{Float64}((6,)) [-2.0, -1.2, -0.4, 0.4, 1.2, 2.0]
    parameters: Parameters
      ν: Float64 0.2
      σ: Float64 0.00025
      tspan: Tuple{Float64,Float64}
        1: Float64 0.0
        2: Float64 1.0
    Δt: Float64 0.0006666666666666666
    t: Array{Float64}((1501,)) [0.0, 0.0006666666666666666, 0.0013333333333333333, 0.002, 0.0026666666666666666, 0.0033333333333333335, 0.004, 0.004666666666666667, 0.005333333333333333, 0.006  …  0.994, 0.9946666666666667, 0.9953333333333333, 0.996, 0.9966666666666667, 0.9973333333333333, 0.998, 0.9986666666666667, 0.99933

Try solving

In [10]:
solve(prob_test)

0.0
step
0.0006666666666666666
step
0.0013333333333333333
step
0.002
step
0.0026666666666666666
step
0.003333333333333333
step
0.004
step
0.004666666666666667
step
0.005333333333333334
step
0.006000000000000001
step
0.006666666666666668
step
0.007333333333333335
step
0.008000000000000002
step
0.008666666666666668
step
0.009333333333333334
step
0.01
step
0.010666666666666666
step
0.011333333333333332
step
0.011999999999999999
step
0.012666666666666665
step
0.01333333333333333
step
0.013999999999999997
step
0.014666666666666663
step
0.015333333333333329
step
0.015999999999999997
step
0.016666666666666663
step
0.01733333333333333
step
0.017999999999999995
step
0.01866666666666666
step
0.019333333333333327
step
0.019999999999999993
step
0.02066666666666666
step
0.021333333333333326
step
0.021999999999999992
step
0.022666666666666658
step
0.023333333333333324
step
0.02399999999999999
step
0.024666666666666656
step
0.025333333333333322
step
0.02599999999999999
step
0.026666666666666655
step


step
0.2766666666666675
step
0.27733333333333415
step
0.2780000000000008
step
0.27866666666666745
step
0.2793333333333341
step
0.28000000000000075
step
0.2806666666666674
step
0.28133333333333405
step
0.2820000000000007
step
0.28266666666666734
step
0.283333333333334
step
0.28400000000000064
step
0.2846666666666673
step
0.28533333333333394
step
0.2860000000000006
step
0.28666666666666724
step
0.2873333333333339
step
0.28800000000000053
step
0.2886666666666672
step
0.28933333333333383
step
0.2900000000000005
step
0.29066666666666713
step
0.2913333333333338
step
0.2920000000000004
step
0.2926666666666671
step
0.2933333333333337
step
0.29400000000000037
step
0.294666666666667
step
0.29533333333333367
step
0.2960000000000003
step
0.29666666666666697
step
0.2973333333333336
step
0.29800000000000026
step
0.2986666666666669
step
0.29933333333333356
step
0.3000000000000002
step
0.30066666666666686
step
0.3013333333333335
step
0.30200000000000016
step
0.3026666666666668
step
0.30333333333333345

step
0.6453333333333363
step
0.646000000000003
step
0.6466666666666697
step
0.6473333333333364
step
0.6480000000000031
step
0.6486666666666698
step
0.6493333333333365
step
0.6500000000000032
step
0.65066666666667
step
0.6513333333333367
step
0.6520000000000034
step
0.6526666666666701
step
0.6533333333333368
step
0.6540000000000035
step
0.6546666666666702
step
0.6553333333333369
step
0.6560000000000036
step
0.6566666666666703
step
0.657333333333337
step
0.6580000000000037
step
0.6586666666666704
step
0.6593333333333371
step
0.6600000000000038
step
0.6606666666666705
step
0.6613333333333372
step
0.6620000000000039
step
0.6626666666666706
step
0.6633333333333373
step
0.664000000000004
step
0.6646666666666707
step
0.6653333333333374
step
0.6660000000000041
step
0.6666666666666708
step
0.6673333333333376
step
0.6680000000000043
step
0.668666666666671
step
0.6693333333333377
step
0.6700000000000044
step
0.6706666666666711
step
0.6713333333333378
step
0.6720000000000045
step
0.672666666666671

step
0.9313333333333524
step
0.9320000000000191
step
0.9326666666666859
step
0.9333333333333526
step
0.9340000000000193
step
0.934666666666686
step
0.9353333333333527
step
0.9360000000000194
step
0.9366666666666861
step
0.9373333333333528
step
0.9380000000000195
step
0.9386666666666862
step
0.9393333333333529
step
0.9400000000000196
step
0.9406666666666863
step
0.941333333333353
step
0.9420000000000197
step
0.9426666666666864
step
0.9433333333333531
step
0.9440000000000198
step
0.9446666666666865
step
0.9453333333333532
step
0.9460000000000199
step
0.9466666666666866
step
0.9473333333333533
step
0.94800000000002
step
0.9486666666666868
step
0.9493333333333535
step
0.9500000000000202
step
0.9506666666666869
step
0.9513333333333536
step
0.9520000000000203
step
0.952666666666687
step
0.9533333333333537
step
0.9540000000000204
step
0.9546666666666871
step
0.9553333333333538
step
0.9560000000000205
step
0.9566666666666872
step
0.9573333333333539
step
0.9580000000000206
step
0.95866666666668

LoadError: UndefVarError: integrator not defined